## 5.horseIDとjockeyIDの取得

In [37]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from tqdm import tqdm
import functions
import numpy as np
import importlib

pd.options.display.max_rows = 5
pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 30

In [ ]:

df = pd.read_csv('2.2.add.日付_df.csv')
race_id_list = df.index.unique()
# indexをstr型に変換
print(type(race_id_list[0]))
# race_id_list = ["2018106086","2018102272","2018102229","2017105400","2018101729","2015105960"]
race_id_list = map(str,race_id_list)
# df

In [ ]:
test = {}
def scrape_race_results(race_id_lists,par_race_results={}):
  race_results = par_race_results
  for _race_id in tqdm(race_id_lists):
    # if _race_id in race_result.keys():
    #   continue
    time.sleep(1)
    try:
      # スクレイピング
      url = "https://race.netkeiba.com/race/result.html?race_id=" + _race_id
      html = requests.get(url)
      html.encoding= "EUC_JP"
      soup = BeautifulSoup(html.text, "html.parser")
      
      # horse_id
      horse_id_list = []      
      horse_a_list = soup.find('table',attrs={"summary":"全着順"}).find_all('a',attrs={"href":re.compile("/horse/")})
      for a in horse_a_list:
        horse_id = re.findall(r'\d+',a["href"])
        horse_id_list.append(horse_id[0])
      Series1 = pd.Series(data = horse_id_list,dtype=str,name="horse_id")
      # jockey_id
      jockey_id_list = []
      jockey_a_list = soup.find('table',attrs={"summary":"全着順"}).find_all('a',attrs={"href":re.compile("/jockey/")})
      for u in jockey_a_list:
        jockey_id = re.findall(r'\d+',u["href"])
        jockey_id_list.append(jockey_id[0])
      Series2 = pd.Series(data = jockey_id_list,dtype=str,name="jockey_id")
      
      results = pd.concat([Series1, Series2], axis=1)
      test[_race_id] = results
      
    except UnicodeDecodeError as e:
      # print(e)
      print(f"レースID: {_race_id} のデータはありません")
      continue
    except KeyboardInterrupt:
      break
    except TimeoutError:
      break
  return test

In [ ]:
results_new = scrape_race_results(race_id_list)
# results_new

----

## 6.取得したデータを成形して元のデータと結合

In [ ]:
for key in results_new.keys():
  results_new[key].index = [key] * len(results_new[key])
print(results_new)
results = pd.concat([results_new[key] for key in results_new.keys()],sort=False)
results

In [ ]:
print(type(results))
print(len(results))
# results.index = results.index.astype(str)
print(type(results.index[0]))
print(type(df))
print(len(df))
df.index = df.index.astype(str)
print(type(df.index[0]))


In [ ]:
# pd.set_option('display.max_rows',None)

df_add_id = pd.concat([df,results],axis=1)

In [ ]:
# df_add_id.to_csv("3.1.df_add_id.csv")

----

## 7.idのリストを作成

In [38]:
horse_id_list = df_add_id["horse_id"].unique()
len(horse_id_list)
# horse_id_list = horse_id_list[:50]
horse_id_list = ["2018106086","2018102272","2018102229","2017105400","2018101729","2015105960"]

In [ ]:
# url = "https://db.netkeiba.com/horse/2019109094"
# url = "https://db.netkeiba.com/horse/2018102444"
# time.sleep(1)
# pd.read_html(url,match='日付')

In [39]:
test = {}
def scrape_horse_results(horse_id_list,par_horse_results={}):
  horse_results = par_horse_results
  for _horse_id in tqdm(horse_id_list):
    # if _horse_id in horse_id_list.keys():
    #   continue
    time.sleep(1)
    try:
      # スクレイピング
      url = "https://db.netkeiba.com/horse/" + _horse_id
      time.sleep(1)
      # [0]をつけてリストの中だけを取得
      horse_results[_horse_id] = pd.read_html(url,match='日付',encoding='EUC-JP')[0]
    except UnicodeDecodeError as e:
      # print(e)
      # print(f"レースID: {race_id} のデータはありません")
      continue
    except KeyboardInterrupt:
      break
        
    except TimeoutError:
      break
    # print(horse_results)
      
  return horse_results

In [40]:
horse_results = scrape_horse_results(horse_id_list)

 83%|████████▎ | 5/6 [00:12<00:02,  2.38s/it]/Users/masa/Desktop/scikit-learn-test/venv39/lib/python3.9/site-packages/bs4/__init__.py:223: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")
100%|██████████| 6/6 [00:15<00:00,  2.50s/it]


In [41]:
horse_results

{'2018106086':             日付    開催 天気  R    レース名  映像  頭数  枠番  馬番   オッズ   人気    着順    騎手  斤量  \
 0   2022/04/04    川崎  雨  8  C2一　二三 NaN  13   5   7  10.3  4.0     2   笹川翼  56   
 1   2022/03/16    浦和  曇  8   C2六　七 NaN   9   4   4   8.6  5.0     1   笹川翼  56   
 ..         ...   ... .. ..     ...  ..  ..  ..  ..   ...  ...   ...   ...  ..   
 14  2020/12/12  6阪神3  晴  3   2歳未勝利 NaN  12   2   2   4.4  2.0     5  福永祐一  55   
 15  2020/11/21  5阪神5  晴  6    2歳新馬 NaN  15   8  16  23.6  7.0  3(降)  鮫島克駿  55   
 
        距離 馬場 馬場指数     タイム   着差 ﾀｲﾑ指数       通過        ペース    上り      馬体重  \
 0   ダ1600  不   **  1:46.0  0.4    **  4-6-2-2  37.6-41.3  41.5  539(-2)   
 1   ダ1400  良   **  1:30.0 -0.3    **  2-2-2-1  37.5-39.4  39.2  541(-3)   
 ..    ... ..  ...     ...  ...   ...      ...        ...   ...      ...   
 14  ダ1200  良   **  1:14.1  0.8    **      4-4  35.4-37.9  38.4   532(0)   
 15  ダ1200  良   **  1:14.1  0.1    **      1-1  35.7-38.3  38.4   532(0)   
 
     厩舎ｺﾒﾝﾄ  備考   勝ち馬(2着馬)     賞金 

----

## 取得したデータを成形

In [42]:
for key in horse_results.keys():
  horse_results[key].index = [key] * len(horse_results[key])
results = pd.concat([horse_results[key] for key in horse_results.keys()],sort=False)

In [43]:
# pd.options.display.max_rows = None

results

,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,オッズ,人気,着順,騎手,斤量,距離,馬場,馬場指数,タイム,着差,ﾀｲﾑ指数,通過,ペース,上り,馬体重,厩舎ｺﾒﾝﾄ,備考,勝ち馬(2着馬),賞金
2018106086,2022/04/04,川崎,雨,8,C2一 二三,NaN,13,5,7,10.3,4.0,2,笹川翼,56,ダ1600,不,**,1:46.0,0.4,**,4-6-2-2,37.6-41.3,41.5,539(-2),NaN,NaN,ゲストプリンシパル,40.0
2018106086,2022/03/16,浦和,曇,8,C2六 七,NaN,9,4,4,8.6,5.0,1,笹川翼,56,ダ1400,良,**,1:30.0,-0.3,**,2-2-2-1,37.5-39.4,39.2,541(-3),NaN,NaN,(プリトヴィツェ),100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015105960,2017/10/29,4東京9,雨,3,2歳未勝利,NaN,13,3,3,15.1,6.0,4,シュミノ,54,芝1600,不,**,1:39.7,0.6,**,3-4,36.9-36.8,36.8,436(+2),NaN,NaN,ミカリーニョ,75.0
2015105960,2017/09/30,4中山8,曇,5,2歳新馬,NaN,15,7,12,71.7,10.0,3,石橋脩,54,芝1600,良,**,1:37.2,0.1,**,3-3-2,36.9-34.6,34.6,434(0),NaN,NaN,デュッセルドルフ,180.0


In [ ]:
# results.to_csv("3.2.horse_vs_data.csv")
results.to_csv("追加.horse_vs_data.csv")

In [ ]:
# time.sleep(2)
# df = pd.read_csv("3.2.horse_vs_data.csv")

In [ ]:
# print(df.columns)
# df = df.iloc[:,:29]
# print(df.columns)

----

## データの自動成形

In [ ]:
importlib.reload(functions)

arr = [
       # 13238,13239,13334,13335,13958,13978,13993,14019,
       # 14020,14316,14317,14341,14342,14388,14548,14549,
       # 14555,14556,14718,14719,14555,14556,15056,15057,
       # 15129,14555,14556,15057,15129,15130,15316,15317,
       # 15378,15379,15425,15426,15466,15517,15518,15519,
       # 15557,15558,15638,15639,15665,15666,15799,15800,
       # 15838,15839,15859,15860
       ]

clean_index = list(map(lambda x: x-1,arr))
# l,l_length,columns_len = Molding(clean_index,"3.2.horse_vs_data.csv")

# l,l_length,columns_len,おかしいリスト = functions.Molding_new(clean_index,"3.4.horse_vs_data.csv")
l,l_length,columns_len,おかしいリスト = functions.Molding_new(clean_index,"追加.horse_vs_data.csv")
おかしいリスト

In [ ]:
print(f"全体のlen{l_length}")
print(f"カラムのlen{columns_len}")

In [ ]:
for i in l :
  if len(i) == 32:pass
  else: print(len(i), i)

In [ ]:

add = np.array(l[1:]).reshape(l_length-1,32)


10秒

In [ ]:
df_add = pd.DataFrame(add, columns=l[0])
# print(df_add)

In [ ]:
# df_add.to_csv('3.4.horse_vs_data.csv',index=False)

In [ ]:
df = pd.read_csv("3.4.horse_vs_data.csv")

In [ ]:
pd.options.display.max_rows = None
for i in df.columns:
  print(df[i].value_counts())

In [ ]:
# df